In [1]:
import os
import time
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from pcdet.models import build_network, load_data_to_gpu
from pcdet.datasets import build_dataloader
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils

/home/pengcheng/anaconda3/envs/openpcdet/lib/python3.8/site-packages/tqdm-4.66.1-py3.8.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configs and checkpoints for all branches
# branches = [['cfgs/waymo_models/centerpoint_dyn_pillar020_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar020_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_pillar024_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar024_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_pillar028_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar028_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_pillar032_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar032_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_pillar036_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar036_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_pillar040_1x.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_pillar040_1x.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0075.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0075.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0100.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0100.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0125.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0125.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0150.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0150.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0175.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0175.pth'],
#             ['cfgs/waymo_models/centerpoint_dyn_voxel0200.yaml',
#                 '../output/waymo_checkpoints/centerpoint_dyn_voxel0200.pth']]

branches = [['cfgs/waymo_models/dsvt_pillar.yaml',
                '../output/waymo_models/dsvt_pillar/default/ckpt/checkpoint_epoch_11.pth']]

In [3]:
# Timing helper function
def time_sync():
    # pytorch-accurate time
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    return time.time()

In [5]:
profiling_results = []
for i, (config, ckpt) in enumerate(branches[6:7]):
    print(i)
    # Read the config file
    cfg_from_yaml_file(config, cfg)
    cfg.TAG = Path(config).stem
    cfg.EXP_GROUP_PATH = '/'.join(config.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
    np.random.seed(1024)
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    dist_test = False
    total_gpus = 1
    batch_size = 1
    workers = 4

    # Create results directory
    output_dir = cfg.ROOT_DIR / 'output' / 'profiling' / cfg.TAG
    output_dir.mkdir(parents=True, exist_ok=True)
    logger = common_utils.create_logger()

    # Build the dataloader
    test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        class_names=cfg.CLASS_NAMES,
        batch_size=batch_size,
        dist=dist_test, workers=workers, logger=logger, training=False)
    print(f'Total number of samples: \t{len(test_set)}')
    subset = []
    for idx, data_dict in enumerate(test_loader):
        subset.append(data_dict)
        if idx == 100:
            break
    #print(subset)

    # Build the model
    model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
    model.load_params_from_file(filename=ckpt, logger=logger)
    model.cuda()
    model.eval()

    # Inference
    class_names = test_set.class_names
    det_annos = []
    e2e_lat = []
    latBreakdown_list = []
    for idx, data_dict in enumerate(tqdm(subset)):
        load_data_to_gpu(data_dict)
        start_time = time_sync()
        with torch.no_grad():
            pred_dicts, _, latBreakdown = model.forward(data_dict)
        inference_time = 1000 * (time_sync() - start_time)
        e2e_lat.append(inference_time)
        annos = test_set.generate_prediction_dicts(data_dict, pred_dicts, class_names, None)
        det_annos += annos

    # Save results: 1. detection results; 2. latency results
    # result_dir = 'results'
    # with open(result_dir / cfg.TAG + '.pkl', 'wb') as f:
    #     pickle.dump(det_annos, f)
    # profiling_results.append([cfg.TAG, e2e_lat])
# np.save('results/latency_profiling_test', profiling_results)

2024-02-02 14:03:07,994   INFO  Loading Waymo dataset
2024-02-02 14:03:07,994   INFO  Loading Waymo dataset


0


2024-02-02 14:03:09,662   INFO  Total skipped info 0
2024-02-02 14:03:09,662   INFO  Total skipped info 0
2024-02-02 14:03:09,663   INFO  Total samples for Waymo dataset: 39987
2024-02-02 14:03:09,663   INFO  Total samples for Waymo dataset: 39987


Total number of samples: 	39987


2024-02-02 14:03:11,134   INFO  ==> Loading parameters from checkpoint ../output/waymo_checkpoints/centerpoint_dyn_voxel0075.pth to GPU
2024-02-02 14:03:11,134   INFO  ==> Loading parameters from checkpoint ../output/waymo_checkpoints/centerpoint_dyn_voxel0075.pth to GPU
2024-02-02 14:03:11,824   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f+py94faff6
2024-02-02 14:03:11,824   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f+py94faff6
2024-02-02 14:03:11,849   INFO  ==> Done (loaded 279/279)
2024-02-02 14:03:11,849   INFO  ==> Done (loaded 279/279)
  0%|          | 0/2 [00:00<?, ?it/s]/home/pengcheng/adaptive-3d-openpcdet/pcdet/models/backbones_3d/vfe/dynamic_mean_vfe.py:68: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, roundin

========Module======= DynamicMeanVFE()
========Module======= VoxelResBackBone8x(
  (conv_input): SparseSequential(
    (0): SubMConv3d(5, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
    (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv1): SparseSequential(
    (0): SparseBasicBlock(
      (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.MaskImplicitGemm)
      (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.MaskImplicitGemm)
      (bn2): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_s

100%|██████████| 2/2 [00:00<00:00,  2.34it/s]

========Module======= HeightCompression()
========Module======= BaseBEVBackbone(
  (blocks): ModuleList(
    (0): Sequential(
      (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      (1): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (5): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (8): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (9): ReLU()
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (11): BatchNorm2d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (12): ReLU()
      (13): Conv2d(128, 1

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
test_set == test_loader.dataset

True